In [8]:
from simple.binance import getHistMonth
from simple.chart import chartFigure, updateFigure
from simple.pretty import pmap
import pandas as pd
import numpy as np
from simdkalman import KalmanFilter

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

In [3]:
plt.rcParams['figure.dpi'] = 80
plt.rcParams['figure.subplot.left'] = 0.04
plt.rcParams['figure.subplot.right'] = 0.96
plt.rcParams['figure.subplot.top'] = 0.96
plt.rcParams['figure.subplot.bottom'] = 0.04
plt.rcParams['axes.grid'] = False
plt.rcParams['grid.color'] = 'lightgray'
plt.rcParams['figure.figsize'] = (14,4)

In [4]:
frame = '1h'

tickers = [
    '1000BTTCUSDT', '1000SHIBUSDT', '1000XECUSDT', '1INCHUSDT', 'AAVEUSDT', 'ADABUSD', 'ADAUSDT', 'AKROUSDT',
    'ALGOUSDT', 'ALICEUSDT', 'ALPHAUSDT', 'ANKRUSDT', 'ANTUSDT', 'API3USDT', 'ARPAUSDT', 'ARUSDT', 'ATAUSDT', 'ATOMUSDT',
    'AUDIOUSDT', 'AVAXUSDT', 'AXSUSDT', 'BAKEUSDT', 'BALUSDT', 'BANDUSDT', 'BATUSDT', 'BCHUSDT', 'BELUSDT', 'BLZUSDT',
    'BNBBUSD', 'BNBUSDT', 'BTCBUSD', 'BTCDOMUSDT', 'BTCSTUSDT', 'BTCUSDT', 'BTSUSDT', 'C98USDT', 'CELOUSDT', 'CELRUSDT',
    'CHRUSDT', 'CHZUSDT', 'COMPUSDT', 'COTIUSDT', 'CRVUSDT', 'CTKUSDT', 'CTSIUSDT', 'CVCUSDT', 'DASHUSDT', 'DEFIUSDT',
    'DENTUSDT', 'DGBUSDT', 'DODOUSDT', 'DOGEBUSD', 'DOGEUSDT', 'DOTUSDT', 'DUSKUSDT', 'DYDXUSDT', 'EGLDUSDT', 'ENJUSDT',
    'ENSUSDT', 'EOSUSDT', 'ETCUSDT', 'ETHBUSD', 'ETHUSDT', 'FILUSDT', 'FLMUSDT', 'FLOWUSDT', 'FTMUSDT', 'FTTBUSD',
    'GALAUSDT', 'GRTUSDT', 'GTCUSDT', 'HBARUSDT', 'HNTUSDT', 'HOTUSDT', 'ICPUSDT', 'ICXUSDT', 'IMXUSDT', 'IOSTUSDT',
    'IOTAUSDT', 'IOTXUSDT', 'KAVAUSDT', 'KLAYUSDT', 'KNCUSDT', 'KSMUSDT', 'LINAUSDT', 'LINKUSDT', 'LITUSDT', 'LPTUSDT',
    'LRCUSDT', 'LTCUSDT', 'LUNAUSDT', 'MANAUSDT', 'MASKUSDT', 'MATICUSDT', 'MKRUSDT', 'MTLUSDT', 'NEARUSDT', 'NEOUSDT',
    'NKNUSDT', 'OCEANUSDT', 'OGNUSDT', 'OMGUSDT', 'ONEUSDT', 'ONTUSDT', 'PEOPLEUSDT', 'QTUMUSDT', 'RAYUSDT', 'REEFUSDT',
    'RENUSDT', 'RLCUSDT', 'ROSEUSDT', 'RSRUSDT', 'RUNEUSDT', 'RVNUSDT', 'SANDUSDT', 'SCUSDT', 'SFPUSDT', 'SKLUSDT',
    'SNXUSDT', 'SOLBUSD', 'SOLUSDT', 'SRMUSDT', 'STMXUSDT', 'STORJUSDT', 'SUSHIUSDT', 'SXPUSDT', 'THETAUSDT', 'TLMUSDT',
    'TOMOUSDT', 'TRBUSDT', 'TRXUSDT', 'UNFIUSDT', 'UNIUSDT', 'VETUSDT', 'WAVESUSDT', 'XEMUSDT', 'XLMUSDT', 'XMRUSDT',
    'XRPBUSD', 'XRPUSDT', 'XTZUSDT', 'YFIIUSDT', 'YFIUSDT', 'ZECUSDT', 'ZENUSDT', 'ZILUSDT', 'ZRXUSDT'
]

In [5]:
def getClose(ticker: str) -> pd.Series:
    return getHistMonth('2022-03', '2022-03', ticker, frame, close_only=True, spot=False)

In [10]:
# parallel load data from binance archive
C = pd.concat(pmap(getClose, tickers, n_jobs=32), axis=1)
C.head()

  0%|          | 0/147 [00:00<?, ?it/s]

,1000BTTCUSDT,1000SHIBUSDT,1000XECUSDT,1INCHUSDT,AAVEUSDT,ADABUSD,ADAUSDT,AKROUSDT,ALGOUSDT,ALICEUSDT,...,XMRUSDT,XRPBUSD,XRPUSDT,XTZUSDT,YFIIUSDT,YFIUSDT,ZECUSDT,ZENUSDT,ZILUSDT,ZRXUSDT
DT,,,,,,,,,,,,,,,,,,,,,
2022-03-01 00:00:00,0.002000,0.026112,0.07946,1.5258,147.71,0.9772,0.9769,0.01429,0.8786,7.787,...,173.42,0.7849,0.7848,3.574,2104.8,21770.0,119.97,39.091,0.04507,0.5846
2022-03-01 01:00:00,0.001979,0.025770,0.07808,1.4963,144.48,0.9671,0.9665,0.01412,0.8621,7.588,...,172.18,0.7711,0.7711,3.523,2084.6,21508.0,119.12,38.791,0.04463,0.5751
2022-03-01 02:00:00,0.001982,0.025634,0.07743,1.4839,144.06,0.9729,0.9722,0.01402,0.8601,7.474,...,171.62,0.7691,0.7687,3.520,2074.2,21374.0,117.72,38.725,0.04481,0.5723
2022-03-01 03:00:00,0.001978,0.025689,0.07754,1.4832,142.36,0.9662,0.9659,0.01397,0.8580,7.456,...,172.63,0.7746,0.7745,3.488,2068.0,21436.0,116.48,38.795,0.04471,0.5725
2022-03-01 04:00:00,0.001979,0.025857,0.07841,1.4977,143.90,0.9654,0.9651,0.01413,0.8671,7.587,...,173.50,0.7769,0.7765,3.531,2090.9,21783.0,117.97,39.293,0.04495,0.5762


In [9]:
kf = KalmanFilter(
    state_transition = np.array([[1,1],[0,1]]),
    process_noise = np.diag([0.2, 0.01])**2,
    observation_model = [[1,0],[1,0]],
    observation_noise = np.eye(2)*3**2)
kf

In [11]:
# smooth and explain existing data
smoothed = kf.smooth(C.values)
smoothed

ValueError: cannot reshape array of size 744 into shape (744,2,1)